<a href="https://colab.research.google.com/github/fitristachan/Air-Quality-Data-Analytic/blob/main/Air_Quality_Data_Analytic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyek Analisis Data: Air Quality Dataset
- **Nama:** Fitri Sagita
- **Email:** fitristarius@gmail.com
- **ID Dicoding:** fitristachan

## Menentukan Pertanyaan Bisnis


- Bagaimana perkembangan parameter kualitas udara di beberapa stasiun pengukuran di Tiongkok selama periode 2013-2017?
- Pada rentang waktu manakah (pagi, siang, sore, atau malam) rata-rata nilai parameter kualitas udara menunjukkan angka tertinggi di Tiongkok selama periode 2013-2017?

## Import Semua Packages/Library yang Digunakan

In [30]:
import os
import pandas as pd
import numpy as np
import geopandas
import matplotlib.pyplot as plt
import scipy as py
import seaborn as sns

## Data Wrangling

### Gathering Data

Karena ada lebih dari satu file dataset dalam folder, maka kita looping terlebih dahulu untuk mendapatkan semua pathfile. Tujuannya agar lebih mudah. kemudian muat tiap file menggunakan .read_csv dan masukkan dataframenya ke dalam list airquality_df_list. Karena tiap tabel memiliki field yang sama dan sudah ada pembedanya di station sebagai penanda darimana data tersebut berasal, maka kita lakukan concat untuk menjadikan semua tabel jadi satu dataframe.

In [31]:

directory = os.fsencode("D:\Chase\Certification-Submission\Air-quality-dataset\Air-Quality-Data-Analytic\PRSA_Data")
airquality_df_list = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"): 
        if isinstance(directory, bytes):
            directory = directory.decode('utf-8')
        data_path = os.path.join(directory, filename)
        df = pd.read_csv(data_path)
        airquality_df_list.append(df)
    else:
        print("Gagal menngambil path file")

airquality_df = pd.concat(airquality_df_list, ignore_index=True)
print(airquality_df)

           No  year  month  day  hour  PM2.5  PM10   SO2   NO2     CO    O3  \
0           1  2013      3    1     0    4.0   4.0   4.0   7.0  300.0  77.0   
1           2  2013      3    1     1    8.0   8.0   4.0   7.0  300.0  77.0   
2           3  2013      3    1     2    7.0   7.0   5.0  10.0  300.0  73.0   
3           4  2013      3    1     3    6.0   6.0  11.0  11.0  300.0  72.0   
4           5  2013      3    1     4    3.0   3.0  12.0  12.0  300.0  72.0   
...       ...   ...    ...  ...   ...    ...   ...   ...   ...    ...   ...   
420763  35060  2017      2   28    19   11.0  32.0   3.0  24.0  400.0  72.0   
420764  35061  2017      2   28    20   13.0  32.0   3.0  41.0  500.0  50.0   
420765  35062  2017      2   28    21   14.0  28.0   4.0  38.0  500.0  54.0   
420766  35063  2017      2   28    22   12.0  23.0   4.0  30.0  400.0  59.0   
420767  35064  2017      2   28    23   13.0  19.0   4.0  38.0  600.0  49.0   

        TEMP    PRES  DEWP  RAIN   wd  WSPM        

**Insight:**
- Dataset yang ada berjumlah 420768 dimulai dari index 0 - 420767
- Temperature ada yang minus dan ada yang plus
- Data yang ada berasal dari tahun 2013 - 2017
- Terdapat 18 kolom, dengan 11 diantaranya merupakan hasil ukur komponen dan partikel yang biasanya menjadi indikator dalam penilaian kualitas udara
- Semua hasil ukur bertipe data float

### Assessing Data

Lihat isi air quality

In [32]:
airquality_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420768 entries, 0 to 420767
Data columns (total 18 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   No       420768 non-null  int64  
 1   year     420768 non-null  int64  
 2   month    420768 non-null  int64  
 3   day      420768 non-null  int64  
 4   hour     420768 non-null  int64  
 5   PM2.5    412029 non-null  float64
 6   PM10     414319 non-null  float64
 7   SO2      411747 non-null  float64
 8   NO2      408652 non-null  float64
 9   CO       400067 non-null  float64
 10  O3       407491 non-null  float64
 11  TEMP     420370 non-null  float64
 12  PRES     420375 non-null  float64
 13  DEWP     420365 non-null  float64
 14  RAIN     420378 non-null  float64
 15  wd       418946 non-null  object 
 16  WSPM     420450 non-null  float64
 17  station  420768 non-null  object 
dtypes: float64(11), int64(5), object(2)
memory usage: 57.8+ MB


Lihat berapa missing value di tiap field dan mengecek ada atau tidaknya duplikasi data

In [33]:
airquality_df.isna().sum()

No             0
year           0
month          0
day            0
hour           0
PM2.5       8739
PM10        6449
SO2         9021
NO2        12116
CO         20701
O3         13277
TEMP         398
PRES         393
DEWP         403
RAIN         390
wd          1822
WSPM         318
station        0
dtype: int64

In [34]:
print("Jumlah duplikasi: ", airquality_df.duplicated().sum())

Jumlah duplikasi:  0


Kode di bawah untuk melihat parameter statistik data dan karena nilai max dalam tiap indikator terlalu besar, sedangkan saya bukan expertise dalam penilaian indikator cuaca dan sumber yang ada di internet pun terbatas, sehingga saya putuskan untuk melakukan grouping nilai maks berdasarkan tiap station untuk dilihat dimanakah nilai maks itu dan apakah ada perbedaan yang sangat signifikan dibanding nilai maks pada station yang lain atau tidak.

In [35]:
airquality_df.describe()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
count,420768.000000,420768.000000,420768.000000,420768.000000,420768.000000,412029.000000,414319.000000,411747.000000,408652.000000,400067.000000,407491.000000,420370.000000,420375.000000,420365.000000,420378.000000,420450.000000
mean,17532.500000,2014.662560,6.522930,15.729637,11.500000,79.793428,104.602618,15.830835,50.638586,1230.766454,57.372271,13.538976,1010.746982,2.490822,0.064476,1.729711
std,10122.116943,1.177198,3.448707,8.800102,6.922195,80.822391,91.772426,21.650603,35.127912,1160.182716,56.661607,11.436139,10.474055,13.793847,0.821004,1.246386
min,1.000000,2013.000000,1.000000,1.000000,0.000000,2.000000,2.000000,0.285600,1.026500,100.000000,0.214200,-19.900000,982.400000,-43.400000,0.000000,0.000000
25%,8766.750000,2014.000000,4.000000,8.000000,5.750000,20.000000,36.000000,3.000000,23.000000,500.000000,11.000000,3.100000,1002.300000,-8.900000,0.000000,0.900000
50%,17532.500000,2015.000000,7.000000,16.000000,11.500000,55.000000,82.000000,7.000000,43.000000,900.000000,45.000000,14.500000,1010.400000,3.100000,0.000000,1.400000
75%,26298.250000,2016.000000,10.000000,23.000000,17.250000,111.000000,145.000000,20.000000,71.000000,1500.000000,82.000000,23.300000,1019.000000,15.100000,0.000000,2.200000
max,35064.000000,2017.000000,12.000000,31.000000,23.000000,999.000000,999.000000,500.000000,290.000000,10000.000000,1071.000000,41.600000,1042.800000,29.100000,72.500000,13.200000


In [36]:
max_per_station = airquality_df.groupby('station').max(numeric_only=True)
print(max_per_station)

                  No  year  month  day  hour  PM2.5   PM10    SO2    NO2  \
station                                                                    
Aotizhongxin   35064  2017     12   31    23  898.0  984.0  341.0  290.0   
Changping      35064  2017     12   31    23  882.0  999.0  310.0  226.0   
Dingling       35064  2017     12   31    23  881.0  905.0  156.0  205.0   
Dongsi         35064  2017     12   31    23  737.0  955.0  300.0  258.0   
Guanyuan       35064  2017     12   31    23  680.0  999.0  293.0  270.0   
Gucheng        35064  2017     12   31    23  770.0  994.0  500.0  276.0   
Huairou        35064  2017     12   31    23  762.0  993.0  315.0  231.0   
Nongzhanguan   35064  2017     12   31    23  844.0  995.0  257.0  273.0   
Shunyi         35064  2017     12   31    23  941.0  999.0  239.0  258.0   
Tiantan        35064  2017     12   31    23  821.0  988.0  273.0  241.0   
Wanliu         35064  2017     12   31    23  957.0  951.0  282.0  264.0   
Wanshouxigon

**Insight:**
- Terdapat banyak missing value di PM2.5, PM10, SO2, NO2, CO, O3, TEMP, PRES, DEWP, RAIN, wd, dan WSPM
- Tidak ada duplikasi data
- Max Nomor tidak sesuai jumlah data sehingga perlu dilakukan penomoran ulang
- SO2 wajar karena dari jurnal yang dirilis tahun 2020 dan berjudul "VERIFIKASI METODE PENENTUAN KADAR SULFUR
DIOKSIDA (SO2) DALAM UDARA AMBIEN SECARA
SPEKTROFOTOMETRI UV-VISIBEL DI PT. KARSA BUANA
LESTARI" oleh DT Metia, link: https://dspace.uii.ac.id/bitstream/handle/123456789/28492/17231059%20Dinda%20Tantri%20Metia.pdf?sequence=1, nilai maksnya adalah 900
- Dilihat dari grouping nilai max yang tidak wajar adalah nilai O3 di Dongsi


### Cleaning Data

Mengganti nilai max dongsi yang abnormal menjadi nilai rata-rata di jam yang sama. Karena setelah dicek ditabel ada beberapa nilai max abnormal yang lebih dari 1000, maka kita tetapkan nilai abnormalnya adalah > 1000.

In [37]:
def replace_dongsi_abnormal(row):
    if row['O3'] >= 1000:
        time_data = airquality_df[(airquality_df['station'] == 'Dongsi') &
                                (airquality_df['month'] == row['month']) & 
                                (airquality_df['day'] == row['day']) & 
                                (airquality_df['hour'] == row['hour'])]
        dongsi_mean = time_data['O3'].mean()
        return dongsi_mean
    else:
        return row['O3']

airquality_df['O3'] = airquality_df.apply(replace_dongsi_abnormal, axis=1)

max_per_station = airquality_df.groupby('station').max(numeric_only=True)
print(max_per_station)

                  No  year  month  day  hour  PM2.5   PM10    SO2    NO2  \
station                                                                    
Aotizhongxin   35064  2017     12   31    23  898.0  984.0  341.0  290.0   
Changping      35064  2017     12   31    23  882.0  999.0  310.0  226.0   
Dingling       35064  2017     12   31    23  881.0  905.0  156.0  205.0   
Dongsi         35064  2017     12   31    23  737.0  955.0  300.0  258.0   
Guanyuan       35064  2017     12   31    23  680.0  999.0  293.0  270.0   
Gucheng        35064  2017     12   31    23  770.0  994.0  500.0  276.0   
Huairou        35064  2017     12   31    23  762.0  993.0  315.0  231.0   
Nongzhanguan   35064  2017     12   31    23  844.0  995.0  257.0  273.0   
Shunyi         35064  2017     12   31    23  941.0  999.0  239.0  258.0   
Tiantan        35064  2017     12   31    23  821.0  988.0  273.0  241.0   
Wanliu         35064  2017     12   31    23  957.0  951.0  282.0  264.0   
Wanshouxigon

Karena dari proses assesing diketahui banyak sekali missing value dan tidak mungkin untuk menghapus semua data yang mengandung missing value karena akan menyebabkan kualitas data menurun, maka missing value tersebut akan diganti dengan menggunakan teknik imputation. Saya menggunakan mean dari tiap group, kecuali wd. Menggunakan mean dari tiap group tujuannya agar nilai pengganti lebih akurat karena merupakan nilai rata-rata dari tiap station.

In [38]:
list_missing = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
for column in list_missing:
    mean_per_column = airquality_df.groupby('station')[column].mean()
    airquality_df[column] = airquality_df.apply(
        lambda row: mean_per_column[row['station']] if pd.isna(row[column]) else row[column],
        axis=1
)

modus_wd_per_station = airquality_df.groupby('station')['wd'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
airquality_df['wd'] = airquality_df.apply(
    lambda row: modus_wd_per_station[row['station']] if pd.isna(row['wd']) else row['wd'],
    axis=1
)

airquality_df.isna().sum()

No         0
year       0
month      0
day        0
hour       0
PM2.5      0
PM10       0
SO2        0
NO2        0
CO         0
O3         0
TEMP       0
PRES       0
DEWP       0
RAIN       0
wd         0
WSPM       0
station    0
dtype: int64

Mengubah kolom nomor yang masih berantakan menjadi berurutan sesuai jumlah data

In [ ]:
airquality_df['No'] = airquality_df.index + 1
airquality_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420768 entries, 0 to 420767
Data columns (total 18 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   No       420768 non-null  int64  
 1   year     420768 non-null  int64  
 2   month    420768 non-null  int64  
 3   day      420768 non-null  int64  
 4   hour     420768 non-null  int64  
 5   PM2.5    420768 non-null  float64
 6   PM10     420768 non-null  float64
 7   SO2      420768 non-null  float64
 8   NO2      420768 non-null  float64
 9   CO       420768 non-null  float64
 10  O3       420768 non-null  float64
 11  TEMP     420768 non-null  float64
 12  PRES     420768 non-null  float64
 13  DEWP     420768 non-null  float64
 14  RAIN     420768 non-null  float64
 15  wd       420768 non-null  object 
 16  WSPM     420768 non-null  float64
 17  station  420768 non-null  object 
dtypes: float64(11), int64(5), object(2)
memory usage: 57.8+ MB


**Insight:**
- Untuk mengisi missing value secara spesifik, di kolom berapa, tidak dapat menggunakan fungsi .fillna. Sehingga saya menggunakan apply, karena datanya tidak mau terisi jika menggunakan .fillna.
- Untuk menemukan modus secara spesifik dari kolom tertentu dengan grouping stasiun juga tidak dapat menggunakan .mode secara langsung dan harus menggunakan aggregate dengan fungsi yang dimau didalamnya.
- Dilakukan replace data abnormal terlebih dahulu baru mengisi missing value karena data abnormal yang ada dapat memengaruhi pengisian missing value dikarenakan nilai valuenya terlalu besar sehingga dapat membuat nilai mean yang dicari membesar, sehingga diketahui urutan cleaning data harus didasarkan pada kebutuhan data
- Missing value dan data abnormal tidak dihapus dengan pertimbangan pertanyaan yang ingin dijawab berhubungan erat dengan waktu yang ada sehingga jika dihapus maka dapat menyebabkan hilangnya nilai di waktu tertentu yang menyebabkan ketidakakuratan hasil.

## Exploratory Data Analysis (EDA)

### Explore ...

**Insight:**
- xxx
- xxx

## Visualization & Explanatory Analysis

### Pertanyaan 1:

### Pertanyaan 2:

**Insight:**
- xxx
- xxx

## Analisis Lanjutan (Opsional)

## Conclusion

- Conclution pertanyaan 1
- Conclution pertanyaan 2